### Importaciones

In [1]:
# Paquetes necesarios para la ejecución del notebook
import win32com.client
import os
import pandas as pd
import openpyxl
import polars as pl
import pygetwindow as gw
from pathlib import Path
import importlib

# Módulos personalizados
import download_mail_files as dmf
import update_local_files as ulf
import update_transportista_resumen_file as utr
import take_powerbi_graphics as tk_pbi
import analysis_and_operations as aos
import create_html_report as chr
import send_email as sm
import log_management as log
import continue_with_report_interface as cwri

### Configuracion

In [2]:
# Constantes usadas en el notebook
MAPI = "MAPI" # Messaging Application Programming Interface
DOT = "."
OUTLOOK = "Outlook"
APPLICATION = "Application"

# Diccionarios
outlook_folder_codes = {
    0: 'Calendario',
    1: 'Contactos',
    2: 'Borradores',
    3: 'Diario / Jornal',
    4: 'Notas',
    5: 'Tareas',
    6: 'Bandeja de entrada',
    7: 'Bandeja de salida',
    8: 'Elementos enviados',
    9: 'Elementos eliminados',
    10: 'Bandeja de correo del servidor',
    11: 'Conflictos',
    12: 'Elementos de sincronizacion local',
    13: 'Elementos de sincronizacion (Envio)',
    14: 'Elementos de sincronización (Recibo)',
    15: 'Elementos de sincronización completa',
    16: 'Diario de formularios',
    17: 'Carpeta de búsqueda',
    18: 'Bandeja para reglas cliente',
    19: 'Carpeta de sugerencias de correo',
}
outlook_object_types = {
    "AppointmentItem": 26,
    "MailItem": 43,
    "TaskItem": 46,
    "ContactItem": 48,
    "MeetingItem": 53,
}
parse_locaciones = {
    '06 AYA EL PEDREGAL': 'Pedregal',
    '38 AYA ATICO': 'Atico',
    '40 AYA CHALA': 'Chala',
    '88 AYA CAMANA': 'Camana'
}
meta = {
    '06 AYA EL PEDREGAL': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 1.2, 2: 1.2, 3: 1.2, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1},
        2025: {1: 0.88, 2: 0.88, 3: 0.88, 4: 0.88, 5: 0.88, 6: 0.88, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    },
    '38 AYA ATICO': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 0.4, 2: 0.4, 3: 0.4, 4: 0.48, 5: 0.48, 6: 0.48, 7: 0.48, 8: 0.48, 9: 0.48, 10: 0.48, 11: 0.48, 12: 0.48},
        2025: {1: 0.24, 2: 0.24, 3: 0.24, 4: 0.24, 5: 0.24, 6: 0.24, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    },
    '40 AYA CHALA': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: .7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 0.5, 2: 0.5, 3: 0.5, 4: 0.60, 5: 0.60, 6: 0.60, 7: 0.60, 8: 0.60, 9: 0.60, 10: 0.60, 11: 0.60, 12: 0.60},
        2025: {1: 0.31, 2: 0.31, 3: 0.31, 4: 0.31, 5: 0.31, 6: 0.31, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    },
    '88 AYA CAMANA': {
        2023: {1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 0.7, 7: 0.7, 8:0.7, 9:0.7, 10:0.7, 11:0.7, 12:0.7},
        2024: {1: 0.85, 2: 0.85, 3: 0.85, 4: 0.85, 5: 0.85, 6: 0.85, 7: 0.85, 8: 0.85, 9: 0.85, 10: 0.85, 11: 0.85, 12: 0.85},
        2025: {1: 0.57, 2: 0.57, 3: 0.57, 4: 0.57, 5: 0.57, 6: 0.57, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0},
    }
}
months = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio',
    7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 
    11: 'Noviembre', 12: 'Diciembre'
}
vendedores = {
    '06 AYA EL PEDREGAL': {
        '0000001013': 'AYALA CCAHUANA EDMUNDO',
        '0000001006': 'BARRAZA REVILLA FREDY',
        '0000001000': 'CARDENAS CHOQUECAJIA WHITNEY AWARD',
        '0000001001': 'CHARA ROJAS GISELA SOLEDAD',
        '0000001007': 'CHINO QUISPE JORGE LUIS',
        '0000006014': 'EDWAR QUISPE CHINO',
        '0000006009': 'GALLARDO HUAYLLA RENATO LEUTERIO',
        '0000006003': 'GALLEGOS NUÑEZ JUAN FREDY',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001010': 'HUANCA MAMANI JACKSON',
        '0000006006': 'HUAYCHO TORRES ULBER',
        '0000006005': 'HUAYCHO TORRES ULBER',
        '0000001012': 'HUAYLLAZO HUACCHA CRISTHIAN PAOLO',
        '0000006012': 'NELSON RAUL CONZA CHARCA',
        '0000001002': 'PACHECO CONDORI ANDRES OSCAR',
        '0000001011': 'QUISPE CHINO WILIAN YURI',
        '0000001004': 'QUISPE HUAYLLA MARISOL S.',
        '0000006013': 'RONALD GONZALO HUILLCA MAMANI',
        '0000001003': 'TACO XESSPE KELY SOFIA',
        '0000001005': 'VALDERRAMA ELLIS JESSICA A.',
        '0000001008': 'VEND NVA RUTA 10',
        '0000001009': 'YUCRA JIMENEZ ANA LUZ',
        '0000001014': 'CHARA ROJAS GISELA SOLEDAD',
        '0000001015': 'BARRAZA REVILLA FREDY',
        '0000001019': 'BATALLANOS SANCA RODRIGO LEOPOLDO',
        '0000001016': 'PARI PUCHO FREDY OSWALDO',
        '0000001017': 'COAGUILLA MAMANI JOSE ALBERTO',
        '0000001018': 'ORTEGA MAMANI JORGE LUIS'
    },
    '38 AYA ATICO': {
        '0000001001': 'ALANYA RAMIREZ FERNANDO JOSUE',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001000': 'SAUL ANDRES VIÑA VIZCARDO'
    },
    '40 AYA CHALA': {
        '0000001004': 'CANALES AGUILAR HILBERTO',
        '0000001007': 'CHIPANA JURADO JHORS EDUARDO',
        '0000006010': 'DIONICIO DANIEL HUARCAYA SALAZAR',
        '0000005999': 'GESTOR VIRTUAL',
        '0000006009': 'GLOBER FELIPE JARA MAQUER',
        '0000006005': 'GONZALES CHURA MAGNO ALFREDO',
        '0000001000': 'HERRERA TAPIA EDWIN DONATO',
        '0000006002': 'JARA CARAZAS RODOLFO JOSUE.L',
        '0000006007': 'JUAN CARLOS ARIAS BENITES',
        '0000001003': 'MAMANI TINTAYA KRISTHOFER',
        '0000001006': 'MAMANI TINTAYA KRISTHOFER',
        '0000001005': 'QUISE CCAPA EVER',
        '0000001002': 'RODRIGUEZ JOSE ANTONIO',
        '0000001001': 'VENDEDOR RT M1'        
    },
    '88 AYA CAMANA': {
        '0000001007': 'AUCAHUAQUI REVILLA DANIEL HITLER',
        '0000001011': 'CARAZAS REZA LUIS ALBERTO',
        '0000001003': 'CONDORCHOA SIERRA NEMECIO',
        '0000006001': 'CONDORCHOA SIERRA NEMESIO JESUS',
        '0000006002': 'DE LA CRUZ CALCINA ELAR',
        '0000006010': 'EDILBERTO RAMIREZ LARICO',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001002': 'HUAMANI RODRIGUEZ YONATAN ANYIMZAN',
        '0000006011': 'JAIME CHAVEZ CONDORI',
        '0000001005': 'LLERENA DE LA CRUZ RICARDO SNEIDER',
        '0000001004': 'MEDINA VELASQUEZ JAVIER ENRIQUE',
        '0000006014': 'MOISES RICHARD CONDORCHOA SIERRA',
        '0000001008': 'MOLLO YUPANQUI JOSE OMAR',
        '0000001001': 'NO APLICA VENDEDOR',
        '0000006008': 'QUISPE CCACHUCO FREDY',
        '0000001010': 'RAMOS MAMANI RUBEN',
        '0000006016': 'RENATO ELEUTERIO GALLARADO HUAYLLA',
        '0000001006': 'SACARI CHOQUEHUANCA WILSON',
        '0000001000': 'SALAZAR HUAMANI SAUL ANTONIO',
        '0000001012': 'VEND RT X1',
        '0000001009': 'VIZCARDO BUSTAMANTE ALBERTH FRANCK'
    }
}
transportistas_code = {
    '06 AYA EL PEDREGAL': {
        'PACHECO CONDORI FERNANDO JOSE': '606002',
        'GALLEGOS NUÑEZ JUAN FREDY': '606003',
        'HUAYCHO TORRES ULBER': '606006',
        'GALLARDO HUAYLLA RENATO LEUTERIO': '606009',
        'NELSON RAUL CONZA CHARCA': '606012',
        'RONALD GONZALO HUILLCA MAMANI': '606013',
        'MERMA CUTI JHON ELVIS': '606008',
        'HUAYLLAZO CHOQUE CAYETANO DONATO': '606005',
        'EDWAR QUISPE CHINO': '606014',
        'ALVARES, HUAMAN EDWIN': '606011',
        'VIRGILIO HUAYCHO TORRES': '606015',
        'CARMELO RAYNALDO LAGOS CHILE': '606017',
        'ROBIN WILFREDO YUCRA SOTO': '606016',
    },
    '38 AYA ATICO': {
        'CARLOS JERRY QUISPE TOHALINO': '3806004',
        'JAMES  ROGER CUTI CUYO': '3806006',
        'LUIS MICHAEL RONDON IZQUIERDO': '3806005',
        'ZAMUDIO,CHACON PERCY LUIS': '3806002'
    },
    '40 AYA CHALA': {
        'GONZALES CHURA MAGNO ALFREDO': '4006005',
        'JARA CARAZAS RODOLFO JOSUE.L': '4006002',
        'JUAN CARLOS ARIAS BENITES': '4006007',
        'GLOBER FELIPE JARA MAQUER': '4006009',
        'DIONICIO DANIEL HUARCAYA SALAZAR': '4006010',
        'VENTA OFICINA': '4006001',
        'PERCY LUIS ZAMUDIO ALVARES': '4006013',
        'JUAN  MARIO PAYPAY  HUAMANI': '4006014'
    },
    '88 AYA CAMANA': {
        'VENTA OFICINA': '4006001',
        'CONDORCHOA SIERRA NEMESIO JESUS': '8806001',
        'DE LA CRUZ CALCINA ELAR': '8806002',
        'RAMOS GUEVARA DEHIVI PATRIPT': '8806003',
        'QUISPE CCACHUCO FREDY': '8806008',
        'EDILBERTO RAMIREZ LARICO': '8806010',
        'JAIME CHAVEZ CONDORI': '8806011',
        'MOISES RICHARD CONDORCHOA SIERRA': '8806014',
        'ALEXI JOSELYN MOGROVEJO HUAMANI': '8806015',
        'MIGUEL ANGEL MESIAS SILVA': '8806013',
        'EVERARDO JESUS NUÑEZ SARAVIA': '8806012',
        'VENTA DE OFICINA': '8808994',
        'RENATO ELEUTERIO GALLARADO HUAYLLA': '8806016'
    },
}
transportista = {
    "name": "transportista",
    "mail_subject": "Reporte de ordenes de carga diario",         # Nombre del asunto de correo
    "local_file_name": "Cf_programadas_por_transportista.csv",    # Nombre del archivo local
    "local_file_address": "",                                     # Direccion del archivo local
    "mail_file_address": "",                                      # Direccion del archivo del correo
    "mail_sheet_name": "Guias",
    "mail_received_time": "",
    "date": "Fecha",
    "date_format": "dd\/mm\/yyyy",
    "transportista": "Código de Transportista",
}
transportista_resumen = {
    "name": "transportista resumen",
    "mail_subject": "Reporte de ordenes de carga diario",                 # Nombre del asunto de correo
    "local_file_name": "Cf_programadas_por_transportista_resumen.csv",    # Nombre del archivo local
    "local_file_address": "",                                             # Direccion del archivo local
    "date": "Fecha",
    "date_format": "dd\/mm\/yyyy",
}
ruta = {
    "name": "ruta",
    "mail_subject": "Venta perdida diaria por cliente y ruta - acum mes",   # Nombre del asunto de correo
    "local_file_name": "Cf_rech_por_ruta.csv",                              # Nombre del archivo local
    "local_file_address": "",                                               # Direccion del archivo local
    "mail_file_address": "",                                                # Direccion del archivo del correo
    "mail_sheet_name": "Motivos_VP (clte)",
    "mail_received_time": "",
    "date": "Día",
    "date_format": "d\/m\/yy",
    "transportista": "Código Transportista",
}

# Listas
locaciones = ['06 AYA EL PEDREGAL', '38 AYA ATICO', '40 AYA CHALA', '88 AYA CAMANA']

# Variables de configuracion
root_address = r'C:\Informacion\rechazos' # Direcccion de carpeta raiz
test_address = r'\prueba'
backup_address = r'\backup'
project_address = Path.cwd()
mail_report_folder_address = "mail_report"

<>:203: SyntaxWarning: invalid escape sequence '\/'
<>:212: SyntaxWarning: invalid escape sequence '\/'
<>:223: SyntaxWarning: invalid escape sequence '\/'
<>:203: SyntaxWarning: invalid escape sequence '\/'
<>:212: SyntaxWarning: invalid escape sequence '\/'
<>:223: SyntaxWarning: invalid escape sequence '\/'
C:\Users\AYACDA23\AppData\Local\Temp\ipykernel_8424\3065988753.py:203: SyntaxWarning: invalid escape sequence '\/'
  "date_format": "dd\/mm\/yyyy",
C:\Users\AYACDA23\AppData\Local\Temp\ipykernel_8424\3065988753.py:212: SyntaxWarning: invalid escape sequence '\/'
  "date_format": "dd\/mm\/yyyy",
C:\Users\AYACDA23\AppData\Local\Temp\ipykernel_8424\3065988753.py:223: SyntaxWarning: invalid escape sequence '\/'
  "date_format": "d\/m\/yy",


### Obtener correos de Outlook

Configuracion de uso de Outlook

In [3]:
# Conectar a Outlook
#outlook_folder_code = int(input(f'{" ".join(["(" + str(key) + ": " + value + ")" for key, value in outlook_folder_codes.items()])}'))
outlook = win32com.client.Dispatch(OUTLOOK+DOT+APPLICATION).GetNamespace(MAPI)

outlook_folder = outlook.GetDefaultFolder(6)
print("Tipo de folder: ", outlook_folder)

Tipo de folder:  Bandeja de entrada


Guardar archivo de outlook

In [4]:
importlib.reload(dmf)

# Buscar el correo más reciente con archivo Excel
mails = outlook_folder.Items

# Ordenar por fecha descendente
mails.Sort("[ReceivedTime]", True) # (mails) Es un objeto lista

''' MAIL PROPERTIES
    | (mail.Subject) (mail.ReceivedTime) (mail.SenderName)       |
    | (mail.SenderEmailAddress) (mail.To) (mail.CC)              |
    | (mail.Body) (mail.Attachments.Count) (mail.CreationTime)   |
    | (mail.LastModificationTime) (mail.EntryID)                 |
'''

(
    transportista['mail_file_address'], 
    transportista['mail_received_time'], 
    ruta['mail_file_address'], 
    ruta['mail_received_time']
) = dmf.download_mail_files(
    mails, 
    outlook_object_types['MailItem'],
    root_address,
    test_address,
    transportista, 
    ruta
)

c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


DATOS: {'transportista': {'file_address': 'C:\\Informacion\\rechazos\\prueba\\Reporte de Guias de Distribución por FechaV3_54_474526182898396756.xlsx', 'received_time': '2025-06-12'}, 'ruta': {'file_address': 'C:\\Informacion\\rechazos\\prueba\\Venta perdida x Cliente y ruta diaria_3726_5070936250992178252.xlsx', 'received_time': '2025-06-12'}}


Establecer configuraciones finales

In [5]:
transportista['local_file_address'] = os.path.join(root_address+test_address, transportista['local_file_name'])
transportista_resumen['local_file_address'] = os.path.join(root_address+test_address, transportista_resumen['local_file_name'])
ruta['local_file_address'] = os.path.join(root_address+test_address, ruta['local_file_name'])

# Configuracion de documentos
def imprimir_diccionario(nombre, diccionario):
    print(f'\n{nombre.upper()}:')
    for clave, valor in diccionario.items():
        print(f'{clave}: {valor}')

imprimir_diccionario('Transportista', transportista)
imprimir_diccionario('Transportista Resumen', transportista_resumen)
imprimir_diccionario('Ruta', ruta)


TRANSPORTISTA:
name: transportista
mail_subject: Reporte de ordenes de carga diario
local_file_name: Cf_programadas_por_transportista.csv
local_file_address: C:\Informacion\rechazos\prueba\Cf_programadas_por_transportista.csv
mail_file_address: C:\Informacion\rechazos\prueba\Reporte de Guias de Distribución por FechaV3_54_474526182898396756.xlsx
mail_sheet_name: Guias
mail_received_time: 2025-06-12
date: Fecha
date_format: dd\/mm\/yyyy
transportista: Código de Transportista

TRANSPORTISTA RESUMEN:
name: transportista resumen
mail_subject: Reporte de ordenes de carga diario
local_file_name: Cf_programadas_por_transportista_resumen.csv
local_file_address: C:\Informacion\rechazos\prueba\Cf_programadas_por_transportista_resumen.csv
date: Fecha
date_format: dd\/mm\/yyyy

RUTA:
name: ruta
mail_subject: Venta perdida diaria por cliente y ruta - acum mes
local_file_name: Cf_rech_por_ruta.csv
local_file_address: C:\Informacion\rechazos\prueba\Cf_rech_por_ruta.csv
mail_file_address: C:\Informac

### Actualizar archivos locales y creacion de resumen

Actualizar Transportista y Ruta

In [6]:
importlib.reload(ulf)

mail_data_is_empty = False

print('[*] Procediendo con la actualización de archivos locales')
transportista_updated, transportista_mail_is_empty = ulf.update_local_file(transportista, locaciones, root_address, test_address, vendedores, transportistas_code)
ruta_updated, ruta_mail_is_empty = ulf.update_local_file(ruta, locaciones, root_address, test_address, vendedores, transportistas_code)

mail_data_is_empty = transportista_mail_is_empty or ruta_mail_is_empty

[*] Procediendo con la actualización de archivos locales


c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


⚠️ No hay datos nuevos en el archivo de correo 'C:\Informacion\rechazos\prueba\Reporte de Guias de Distribución por FechaV3_54_474526182898396756.xlsx' para la hoja 'Guias'.


c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


⚠️ No hay datos nuevos en el archivo de correo 'C:\Informacion\rechazos\prueba\Venta perdida x Cliente y ruta diaria_3726_5070936250992178252.xlsx' para la hoja 'Motivos_VP (clte)'.


c:\Informacion\rechazos\Analisis\automatizacion_reporte_rechazos\update_local_files.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_local_copy[document['date']+"2"] = pd.to_datetime(df_local_copy[document['date']], dayfirst=True, errors='coerce')


Creacion de resumen de Transportista

In [7]:
importlib.reload(utr)

transportista_resumen_updated = utr.create_transportista_resumen_file(meta, transportista_updated)

Guardar archivos

In [8]:
importlib.reload(log)
importlib.reload(ulf)

log.delete_log()
log.write_log(f'Fecha de correos: ({transportista["mail_received_time"]})')

if mail_data_is_empty:
    print(f"❌ Archivos de correo desactualizados (NO PROCEDE EL GUARDADO DE ARCHIVOS)")
    print(f"Archivo de transportista: {transportista['mail_received_time']}")
    print(f"Archivo de ruta: {ruta['mail_received_time']}")
    
    log.write_log('[X] Archivos de correo desactualizados')

else:
    print(f'✅ Archivos de correo actualizados ({transportista["mail_received_time"]})')
    print('[*] Procediendo con el guardado de archivos locales ...')

    ulf.save_local_file_changes(transportista, os.path.join(root_address, test_address+backup_address))
    ulf.save_local_file_changes(ruta, os.path.join(root_address, test_address+backup_address))
    ulf.save_local_file_changes(transportista_resumen, os.path.join(root_address, test_address+backup_address))
    
    ulf.save_local_file_changes(transportista_updated, transportista)
    ulf.save_local_file_changes(ruta_updated, ruta)
    ulf.save_local_file_changes(transportista_resumen_updated, transportista_resumen)

    log.write_log('[✓] Archivos de correo actualizados')

❌ Archivos de correo desactualizados (NO PROCEDE EL GUARDADO DE ARCHIVOS)
Archivo de transportista: 2025-06-12
Archivo de ruta: 2025-06-12


### Envio de reporte

In [23]:
# Fecha de correos
year, month, day = transportista["mail_received_time"].split('-')

### Automatizacion de envio de reporte

In [20]:
# Configuracion usadas para el uso de selenium y chromedriver
METAS_Y_RESUMEN_PAGE = {
    'page_name': 'METAS Y RESUMEN',
    'page_url': 'https://app.powerbi.com/groups/me/reports/73309ec0-8d79-4111-ac74-acee01ed5375/ReportSectionf1e1f415e4cba9cc4035',
    'page_graphics': {
        1: "% CF Rechazadas",
        2: "Total CF Prog, Rech.",
        3: "Resumen de Rechazos - Meta" 
    },    
    'filter_report_by': 'month',
    'join_report_images': True,
}
DETALLES_PAGE = {
    'page_name': 'DETALLES',
    'page_url': 'https://app.powerbi.com/groups/me/reports/73309ec0-8d79-4111-ac74-acee01ed5375/ReportSection3f59cb816b6f52a24171',
    'page_graphics': {
        1: "Detalle de Cajas Físicas Rechazadas",
    },    
    'filter_report_by': 'locacion',
    'join_report_images': False,
}
RECHAZOS_PAGE = {
    'page_name': 'RECHAZOS',
    'page_url': 'https://app.powerbi.com/groups/me/reports/73309ec0-8d79-4111-ac74-acee01ed5375/ReportSection487ba5c07a522bf0fc8c',
    'page_graphics': {
        1: "% CF Rechazadas",
        2: "CF Rech. por Motivo",
        3: "CF Rech. por Transportista",
        4: "Historico % Rechazo por día"
    },
    'filter_report_by': 'locacion',
    'join_report_images': True,
}
PAGES_REPORT = [METAS_Y_RESUMEN_PAGE, DETALLES_PAGE, RECHAZOS_PAGE]
MAIL_TO = "admpedregal@ayacda.com;admcamana@ayacda.com;admchala@ayacda.com;admatico@ayacda.com;suppedregal@ayacda.com;supcamana@ayacda.com;supchalatico@ayacda.com;rgallegos@aclogistica.pe;admcamana@ayacda.com;suppedregal@ayacda.com;admpedregal@ayacda.com;suplajoya@ayacda.com;gersrpb@gmail.com; gportocarrerob@unsa.edu.pe;"
MAIL_CC = "contabilidad@ayacda.com;adm@ayacda.com;gcamana@ayacda.com"

In [ ]:
importlib.reload(tk_pbi)
importlib.reload(chr)
importlib.reload(aos)
importlib.reload(sm)
importlib.reload(cwri)

# if continue_with_report == 'si':

#     # Automatizacion de capturas de graficos del powerbi
#     print(f'\n✅ Archivos de correo con informacion nueva ({transportista["mail_received_time"]})')
#     print('[*] Procediendo con la captura de graficos ...')
#     tk_pbi.take_powerbi_graphics_main(locaciones, PAGES_REPORT)

#     # Automatizacion de generacion de html
#     print(f'\n✅ Archivos de correo correctamente sincronizados ({transportista["mail_received_time"]})')
#     print('[*] Procediendo con el tratamiento de datos html')
#     calculations, total_rechazos = aos.make_calculations_for_locations(root_address, test_address, locaciones, transportista, ruta, month, year)
#     chr.create_html_report_main(mail_report_folder_address, locaciones, calculations, month, year, total_rechazos, months)

#     # Automatizacion de envio de correo
#     print(f'✅ Archivos de correo correctamente sincronizados ({transportista["mail_received_time"]})')
#     print('[*] Procediendo con el tratamiento de datos html')
#     sm.send_email_main(mail_report_folder_address, project_address, month[month], year, MAIL_TO, MAIL_CC)

# else:
#     print(f"❌ No se procedio al proceso automatizado de envio de reporte")

def ejecutar_proceso_automatico():
    # Automatizacion de capturas de graficos del powerbi
    print(f'\n✅ Archivos de correo con información nueva ({transportista["mail_received_time"]})')
    print('[*] Procediendo con la captura de gráficos ...')
    tk_pbi.take_powerbi_graphics_main(locaciones, PAGES_REPORT, mail_report_folder_address)
    print(f"[✓] Captura de graficos terminado\n")

    # Automatizacion de generacion de html
    print('[*] Procediendo con el tratamiento de datos html')
    calculations, total_rechazos = aos.make_calculations_for_locations(
        root_address, test_address, locaciones, transportista, ruta, month, year)
    chr.create_html_report_main(mail_report_folder_address, locaciones, calculations, month, year, total_rechazos, months)
    print(f"[✓] Generacion de HTML terminado\n")

    # Automatizacion de envio de correo
    print('[*] Procediendo con el envío de correo')
    sm.send_email_main(mail_report_folder_address, project_address, months[int(month)], year, MAIL_TO, MAIL_CC)
    print(f"[✓] Envio de correo terminado")

# 🔁 Bucle de confirmación
while True:
    respuesta = cwri.main_interface('Continuar con el reporte?')

    if respuesta == 'sí':
        ejecutar_proceso_automatico()
    else:
        print("\n[✓] Proceso finalizado.")
        break  # salir del bucle

[*] Procediendo con el tratamiento de datos html
[*] Procediendo con el envío de correo
❌ Proceso finalizado por el usuario.
